# Lab 5: Phân loại Văn bản với RNN/LSTM
Thực hiện đầy đủ 5 nhiệm vụ và so sánh các phương pháp


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, f1_score
from gensim.models import Word2Vec
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings('ignore')

# BƯỚC 0: THIẾT LẬP MÔI TRƯỜNG VÀ TẢI DỮ LIỆU

In [4]:
# Đọc dữ liệu
df_train = pd.read_csv(r'D:\10. ky1nam4\NLP\data\hwu\train.csv', sep=',', header=None, names=['text', 'intent'], skiprows=1)
df_val = pd.read_csv(r'D:\10. ky1nam4\NLP\data\hwu\val.csv', sep=',', header=None, names=['text', 'intent'], skiprows=1)
df_test = pd.read_csv(r'D:\10. ky1nam4\NLP\data\hwu\test.csv', sep=',', header=None, names=['text', 'intent'], skiprows=1)

print(f"Train shape: {df_train.shape}")
print(f"Validation shape: {df_val.shape}")
print(f"Test shape: {df_test.shape}")
print(f"\nSố lớp intent: {df_train['intent'].nunique()}")
print("\nVí dụ dữ liệu:")
print(df_train.head())

# Mã hóa nhãn
label_encoder = LabelEncoder()
label_encoder.fit(pd.concat([df_train['intent'], df_val['intent'], df_test['intent']]))

y_train = label_encoder.transform(df_train['intent'])
y_val = label_encoder.transform(df_val['intent'])
y_test = label_encoder.transform(df_test['intent'])

num_classes = len(label_encoder.classes_)
print(f"\nSố lớp sau encoding: {num_classes}")

Train shape: (8954, 2)
Validation shape: (1076, 2)
Test shape: (1076, 2)

Số lớp intent: 64

Ví dụ dữ liệu:
                                                text       intent
0                what alarms do i have set right now  alarm_query
1                    checkout today alarm of meeting  alarm_query
2                              report alarm settings  alarm_query
3  see see for me the alarms that you have set to...  alarm_query
4                       is there an alarm for ten am  alarm_query

Số lớp sau encoding: 64


# NHIỆM VỤ 1: TF-IDF + Logistic Regression

In [5]:
# Tạo pipeline
tfidf_lr_pipeline = make_pipeline(
    TfidfVectorizer(max_features=5000, ngram_range=(1, 2)),
    LogisticRegression(max_iter=1000, random_state=42)
)

# Huấn luyện
print("Đang huấn luyện mô hình TF-IDF + LR...")
tfidf_lr_pipeline.fit(df_train['text'], y_train)

# Dự đoán và đánh giá
y_pred_tfidf = tfidf_lr_pipeline.predict(df_test['text'])
f1_tfidf = f1_score(y_test, y_pred_tfidf, average='macro')

print(f"\nF1-score (Macro) trên tập test: {f1_tfidf:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_tfidf, 
                          target_names=label_encoder.classes_))

Đang huấn luyện mô hình TF-IDF + LR...

F1-score (Macro) trên tập test: 0.8289

Classification Report:
                          precision    recall  f1-score   support

             alarm_query       0.95      0.95      0.95        19
            alarm_remove       1.00      0.73      0.84        11
               alarm_set       0.85      0.89      0.87        19
       audio_volume_down       1.00      0.75      0.86         8
       audio_volume_mute       0.92      0.80      0.86        15
         audio_volume_up       1.00      1.00      1.00        13
          calendar_query       0.55      0.58      0.56        19
         calendar_remove       0.78      0.95      0.86        19
            calendar_set       0.87      0.68      0.76        19
          cooking_recipe       0.92      0.63      0.75        19
        datetime_convert       0.78      0.88      0.82         8
          datetime_query       0.71      0.89      0.79        19
        email_addcontact       0.88   

# NHIỆM VỤ 2: Word2Vec (Trung bình) + Dense Layer

In [7]:
sentences_train = [text.lower().split() for text in df_train['text']]
sentences_val = [text.lower().split() for text in df_val['text']]
sentences_test = [text.lower().split() for text in df_test['text']]

print("Đang huấn luyện Word2Vec...")
w2v_model = Word2Vec(sentences=sentences_train, 
                     vector_size=100, 
                     window=5, 
                     min_count=1, 
                     workers=4,
                     epochs=10,
                     seed=42)

def sentence_to_avg_vector(tokens, model):
    vectors = []
    for word in tokens:
        if word in model.wv:
            vectors.append(model.wv[word])
    
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    
    return np.mean(vectors, axis=0)

print("Đang chuyển đổi câu thành vector...")
X_train_avg = np.array([sentence_to_avg_vector(sent, w2v_model) 
                        for sent in sentences_train])
X_val_avg = np.array([sentence_to_avg_vector(sent, w2v_model) 
                      for sent in sentences_val])
X_test_avg = np.array([sentence_to_avg_vector(sent, w2v_model) 
                       for sent in sentences_test])

w2v_model_nn = Sequential([
    Dense(128, activation='relu', input_shape=(w2v_model.vector_size,)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

w2v_model_nn.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("\nKiến trúc mô hình Word2Vec + Dense:")
w2v_model_nn.summary()

print("\nĐang huấn luyện...")
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history_w2v = w2v_model_nn.fit(
    X_train_avg, y_train,
    validation_data=(X_val_avg, y_val),
    epochs=30,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

test_loss_w2v, test_acc_w2v = w2v_model_nn.evaluate(X_test_avg, y_test, verbose=0)
y_pred_w2v = np.argmax(w2v_model_nn.predict(X_test_avg, verbose=0), axis=1)
f1_w2v = f1_score(y_test, y_pred_w2v, average='macro')

print(f"\nTest Loss: {test_loss_w2v:.4f}")
print(f"Test Accuracy: {test_acc_w2v:.4f}")
print(f"F1-score (Macro): {f1_w2v:.4f}")

Đang huấn luyện Word2Vec...
Đang chuyển đổi câu thành vector...

Kiến trúc mô hình Word2Vec + Dense:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        12,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         4,160 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,344 (99.00 KB)

 Trainable params: 25,344 (99.00 KB)

 Non-trainable params: 0 (0.00 B)


Đang huấn luyện...
Epoch 1/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0378 - loss: 4.0397 - val_accuracy: 0.1403 - val_loss: 3.2974
Epoch 2/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1223 - loss: 3.3580 - val_accuracy: 0.2165 - val_loss: 2.9505
Epoch 3/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1580 - loss: 3.1015 - val_accuracy: 0.2519 - val_loss: 2.7633
Epoch 4/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1902 - loss: 2.9399 - val_accuracy: 0.2797 - val_loss: 2.6201
Epoch 5/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2144 - loss: 2.8108 - val_accuracy: 0.3243 - val_loss: 2.4961
Epoch 6/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2471 - loss: 2.6928 - val_accuracy: 0.3401 - val_loss: 2.4044
Epoch 7/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2702 - loss: 2.5903 - val_accuracy: 0.3652 - val_loss: 2.3451
Epoch 8/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2829 - loss: 2.554

# NHIỆM VỤ 3: Embedding Pre-trained + LSTM

In [8]:
vocab_size = 5000
max_len = 50

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<UNK>")
tokenizer.fit_on_texts(df_train['text'])

train_sequences = tokenizer.texts_to_sequences(df_train['text'])
val_sequences = tokenizer.texts_to_sequences(df_val['text'])
test_sequences = tokenizer.texts_to_sequences(df_test['text'])

X_train_pad = pad_sequences(train_sequences, maxlen=max_len, padding='post')
X_val_pad = pad_sequences(val_sequences, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(test_sequences, maxlen=max_len, padding='post')

print(f"Shape sau padding: {X_train_pad.shape}")

# actual_vocab_size = min(vocab_size, len(tokenizer.word_index) + 1)
actual_vocab_size = max([np.max(X_train_pad), np.max(X_val_pad), np.max(X_test_pad)]) + 1
embedding_dim = w2v_model.vector_size

embedding_matrix = np.zeros((actual_vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < actual_vocab_size and word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

print(f"Embedding matrix shape: {embedding_matrix.shape}")
words_found = np.sum(np.any(embedding_matrix != 0, axis=1))
print(f"Từ tìm thấy trong Word2Vec: {words_found}/{actual_vocab_size}")

lstm_model_pretrained = Sequential([
    Embedding(
        input_dim=actual_vocab_size,
        output_dim=embedding_dim,
        weights=[embedding_matrix],
        input_length=max_len,
        trainable=True
    ),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

lstm_model_pretrained.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("\nKiến trúc mô hình LSTM Pre-trained:")
lstm_model_pretrained.summary()

# Huấn luyện
print("\nĐang huấn luyện...")
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history_lstm_pre = lstm_model_pretrained.fit(
    X_train_pad, y_train,
    validation_data=(X_val_pad, y_val),
    epochs=30,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

# Đánh giá
test_loss_lstm_pre, test_acc_lstm_pre = lstm_model_pretrained.evaluate(
    X_test_pad, y_test, verbose=0)
y_pred_lstm_pre = np.argmax(lstm_model_pretrained.predict(X_test_pad, verbose=0), axis=1)
f1_lstm_pre = f1_score(y_test, y_pred_lstm_pre, average='macro')

print(f"\nTest Loss: {test_loss_lstm_pre:.4f}")
print(f"Test Accuracy: {test_acc_lstm_pre:.4f}")
print(f"F1-score (Macro): {f1_lstm_pre:.4f}")

Shape sau padding: (8954, 50)
Embedding matrix shape: (4265, 100)
Từ tìm thấy trong Word2Vec: 4197/4265

Kiến trúc mô hình LSTM Pre-trained:


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │       426,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 426,500 (1.63 MB)

 Trainable params: 426,500 (1.63 MB)

 Non-trainable params: 0 (0.00 B)


Đang huấn luyện...
Epoch 1/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - accuracy: 0.0165 - loss: 4.1493 - val_accuracy: 0.0177 - val_loss: 4.1306
Epoch 2/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.0175 - loss: 4.1405 - val_accuracy: 0.0177 - val_loss: 4.1258
Epoch 3/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.0165 - loss: 4.1336 - val_accuracy: 0.0177 - val_loss: 4.1256
Epoch 4/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.0186 - loss: 4.1356 - val_accuracy: 0.0177 - val_loss: 4.1258
Epoch 5/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.0188 - loss: 4.1330 - val_accuracy: 0.0177 - val_loss: 4.1248
Epoch 6/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.0176 - loss: 4.1341 - val_accuracy: 0.0177 - val_loss: 4.1242
Epoch 7/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.0159 - loss: 4.1318 - val_accuracy: 0.0177 - val_loss: 4.1243
Epoch 8/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.0223 - los

# NHIỆM VỤ 4: Embedding học từ đầu + LSTM

In [9]:
lstm_model_scratch = Sequential([
    Embedding(
        input_dim=actual_vocab_size,
        output_dim=100,
        input_length=max_len
        # trainable=True (mặc định)
    ),
    LSTM(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

lstm_model_scratch.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Kiến trúc mô hình LSTM từ đầu:")
lstm_model_scratch.summary()

# Huấn luyện
print("\nĐang huấn luyện...")
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history_lstm_scratch = lstm_model_scratch.fit(
    X_train_pad, y_train,
    validation_data=(X_val_pad, y_val),
    epochs=30,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

# Đánh giá
test_loss_lstm_scratch, test_acc_lstm_scratch = lstm_model_scratch.evaluate(
    X_test_pad, y_test, verbose=0)
y_pred_lstm_scratch = np.argmax(lstm_model_scratch.predict(X_test_pad, verbose=0), axis=1)
f1_lstm_scratch = f1_score(y_test, y_pred_lstm_scratch, average='macro')

print(f"\nTest Loss: {test_loss_lstm_scratch:.4f}")
print(f"Test Accuracy: {test_acc_lstm_scratch:.4f}")
print(f"F1-score (Macro): {f1_lstm_scratch:.4f}")

Kiến trúc mô hình LSTM từ đầu:


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


Đang huấn luyện...
Epoch 1/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 10s 26ms/step - accuracy: 0.0152 - loss: 4.1534 - val_accuracy: 0.0177 - val_loss: 4.1348
Epoch 2/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.0142 - loss: 4.1389 - val_accuracy: 0.0177 - val_loss: 4.1282
Epoch 3/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.0165 - loss: 4.1389 - val_accuracy: 0.0177 - val_loss: 4.1263
Epoch 4/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.0201 - loss: 4.1344 - val_accuracy: 0.0177 - val_loss: 4.1270
Epoch 5/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.0159 - loss: 4.1346 - val_accuracy: 0.0177 - val_loss: 4.1253
Epoch 6/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.0157 - loss: 4.1340 - val_accuracy: 0.0177 - val_loss: 4.1247
Epoch 7/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.0171 - loss: 4.1319 - val_accuracy: 0.0177 - val_loss: 4.1243
Epoch 8/30
280/280 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.0183 - lo

# PHÂN TÍCH ĐỊNH TÍNH - Kiểm tra trên các câu khó

In [10]:
test_samples = [
    "can you remind me to not call my mom",
    "is it going to be sunny or rainy tomorrow",
    "find a flight from new york to london but not through paris"
]

print("\nKiểm tra dự đoán trên các câu khó:\n")

for i, sample in enumerate(test_samples, 1):
    print(f"\n{'='*70}")
    print(f"Câu {i}: '{sample}'")
    print(f"{'='*70}")
    
    # Chuẩn bị input cho từng mô hình
    # TF-IDF
    pred_tfidf = tfidf_lr_pipeline.predict([sample])[0]
    
    # Word2Vec
    tokens = sample.lower().split()
    vec = sentence_to_avg_vector(tokens, w2v_model).reshape(1, -1)
    pred_w2v = np.argmax(w2v_model_nn.predict(vec, verbose=0))
    
    # LSTM models
    seq = tokenizer.texts_to_sequences([sample])
    seq_pad = pad_sequences(seq, maxlen=max_len, padding='post')
    pred_lstm_pre = np.argmax(lstm_model_pretrained.predict(seq_pad, verbose=0))
    pred_lstm_scratch = np.argmax(lstm_model_scratch.predict(seq_pad, verbose=0))
    
    print(f"TF-IDF + LR: {label_encoder.classes_[pred_tfidf]}")
    print(f"Word2Vec + Dense: {label_encoder.classes_[pred_w2v]}")
    print(f"LSTM (Pre-trained): {label_encoder.classes_[pred_lstm_pre]}")
    print(f"LSTM (Scratch): {label_encoder.classes_[pred_lstm_scratch]}")



Kiểm tra dự đoán trên các câu khó:


Câu 1: 'can you remind me to not call my mom'
TF-IDF + LR: calendar_set
Word2Vec + Dense: social_post
LSTM (Pre-trained): general_dontcare
LSTM (Scratch): play_audiobook

Câu 2: 'is it going to be sunny or rainy tomorrow'
TF-IDF + LR: weather_query
Word2Vec + Dense: weather_query
LSTM (Pre-trained): general_dontcare
LSTM (Scratch): play_audiobook

Câu 3: 'find a flight from new york to london but not through paris'
TF-IDF + LR: transport_query
Word2Vec + Dense: email_sendemail
LSTM (Pre-trained): general_dontcare
LSTM (Scratch): play_audiobook


# Báo Cáo Lab 5: Phân loại Văn bản với RNN/LSTM

# Báo Cáo Lab 5: Phân loại Văn bản với RNN/LSTM

## 1. Tổng Quan Kết Quả Định Lượng

| Pipeline | F1-score (Macro) | Test Loss | Test Accuracy |
|----------|------------------|-----------|---------------|
| TF-IDF + Logistic Regression | **0.83** | N/A | **0.83** |
| Word2Vec (Avg) + Dense | 0.41 | 1.94 | 0.46 |
| Embedding (Pre-trained) + LSTM | 0.0005 | 4.12 | 0.02 |
| Embedding (Scratch) + LSTM | 0.0005 | 4.12 | 0.02 |

### Nhận Xét Chung về Kết Quả

Kết quả thí nghiệm cho thấy một bức tranh **ngược lại hoàn toàn** với kỳ vọng lý thuyết:
- **TF-IDF + LR đạt hiệu suất tốt nhất** (F1: 0.83)
- **Các mô hình LSTM hoàn toàn thất bại** (F1 ≈ 0, chỉ dự đoán 1-2 lớp)
- Word2Vec + Dense đạt kết quả trung bình (F1: 0.41)

---

## 2. Phân Tích Định Tính - Dự Đoán trên Câu Khó

### **Câu 1: "can you remind me to NOT call my mom"**
**Nhãn thực tế mong đợi:** `reminder_create`

| Mô hình | Dự đoán | Đánh giá |
|---------|---------|----------|
| TF-IDF + LR | `calendar_set` | Sai nhưng gần (cùng về quản lý thời gian) |
| Word2Vec + Dense | `social_post` | Sai hoàn toàn |
| LSTM (Pre-trained) | `general_dontcare` | Sai - mô hình bị collapse |
| LSTM (Scratch) | `play_audiobook` | Sai - mô hình bị collapse |

**Phân tích chi tiết:**
- Câu này có **phụ thuộc xa quan trọng**: từ "NOT" ở giữa câu phủ định hành động "call"
- **Ngữ cảnh phức tạp**: "remind to NOT do X" khác hoàn toàn với "remind to do X"
- **Kỳ vọng:** LSTM với khả năng xử lý chuỗi nên hiểu được mối quan hệ này
- **Thực tế:** LSTM thất bại hoàn toàn, không học được pattern nào
- **TF-IDF tốt hơn:** Dù không hiểu phủ định, nhưng các từ "remind", "me" đủ mạnh để phân loại vào nhóm task management

---

### **Câu 2: "is it going to be sunny OR rainy tomorrow"**
**Nhãn thực tế mong đợi:** `weather_query`

| Mô hình | Dự đoán | Đánh giá |
|---------|---------|----------|
| TF-IDF + LR | `weather_query` |**ĐÚNG** |
| Word2Vec + Dense | `weather_query` |**ĐÚNG** |
| LSTM (Pre-trained) | `general_dontcare` |Sai - mô hình bị collapse |
| LSTM (Scratch) | `play_audiobook` |Sai - mô hình bị collapse |

**Phân tích chi tiết:**
- Câu này có **từ khóa rõ ràng**: "sunny", "rainy", "tomorrow", "weather"
- **Cấu trúc OR:** "sunny OR rainy" cần hiểu logic chọn lựa
- **Kỳ vọng:** Cả bag-of-words và LSTM đều nên đoán đúng
- **Thực tế:** Chỉ TF-IDF và Word2Vec thành công
- **Nguyên nhân thành công của baseline:** Các từ domain-specific ("sunny", "rainy") là strong signals

---

### **Câu 3: "find a flight from new york to london BUT NOT through paris"**
**Nhãn thực tế mong đợi:** `transport_query` hoặc `flight_search`

| Mô hình | Dự đoán | Đánh giá |
|---------|---------|----------|
| TF-IDF + LR | `transport_query` | **ĐÚNG** hoặc gần đúng |
| Word2Vec + Dense | `email_sendemail` | Sai hoàn toàn |
| LSTM (Pre-trained) | `general_dontcare` | Sai - mô hình bị collapse |
| LSTM (Scratch) | `play_audiobook` | Sai - mô hình bị collapse |

**Phân tích chi tiết:**
- Đây là **câu phức tạp nhất** với:
  - Phụ thuộc xa: "BUT NOT through paris" phủ định điều kiện ở cuối
  - Ngữ cảnh đa thành phần: origin, destination, constraint
  - Yêu cầu hiểu logic điều kiện
- **Kỳ vọng:** LSTM với khả năng xử lý chuỗi dài nên vượt trội
- **Thực tế:** LSTM thất bại thảm hại
- **TF-IDF thành công:** Từ "flight", "new york", "london" đủ mạnh để phân loại đúng
- **Word2Vec sai nghiêm trọng:** Có thể do vector trung bình làm mất đi cấu trúc câu

---

## 3. Phân Tích Nguyên Nhân LSTM Thất Bại

### 3.1. Hiện Tượng "Model Collapse"

Cả hai mô hình LSTM đều cho thấy dấu hiệu **model collapse**:
- **Accuracy cực thấp** (~1.8%): Gần như random guessing (1/64 classes ≈ 1.56%)
- **Dự đoán chỉ 1-2 lớp:** "general_dontcare" và "play_audiobook" xuất hiện liên tục
- **Loss cực cao** (4.12): Mô hình không học được gì

### 3.2. Nguyên Nhân Có Thể

#### **A. Vấn đề về Dữ liệu**
1. **Kích thước tập train quá nhỏ** 
   - LSTM cần nhiều dữ liệu hơn nhiều so với phương pháp truyền thống
   - TF-IDF + LR có thể học tốt với ít dữ liệu hơn

2. **Mất cân bằng lớp nghiêm trọng**
   - Một số lớp có quá ít mẫu
   - LSTM có thể học bias về các lớp dominant

3. **Chất lượng tokenization**
   - Có thể vocab_size=5000 quá nhỏ
   - OOV (Out-of-Vocabulary) words không được xử lý tốt

#### **B. Vấn đề về Kiến trúc & Huấn luyện**

1. **Embedding Matrix không hiệu quả**
   ```
   Từ tìm thấy trong Word2Vec: X/Y (tỷ lệ thấp?)
   ```
   - Nhiều từ trong test set không có trong Word2Vec
   - Embedding pre-trained không trainable → không thể adapt

2. **Hyperparameters không phù hợp**
   - LSTM 128 units có thể quá lớn cho dataset nhỏ
   - Dropout 0.2 có thể chưa đủ
   - Batch size 32 có thể không tối ưu
   - Learning rate mặc định có thể không phù hợp

3. **Vanishing Gradient vẫn xảy ra**
   - Dù LSTM được thiết kế để giải quyết vấn đề này
   - Với max_len=50, vẫn có thể gặp khó khăn với phụ thuộc xa

4. **Khởi tạo trọng số không tốt**
   - Embedding từ Word2Vec có thể không phù hợp với domain
   - Khởi tạo ngẫu nhiên (scratch) càng tệ hơn với data ít
